# Powershell Log

## Loading Events

In [1]:
import pandas as pd
import re

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.set_option('display.max_colwidth', None)

df = pd.read_csv(
    "powershell.csv",
    sep=",",
    header=0,
    names=[ "Level", "DateTime", "Origin", "EventID", "Task", "Category" ],
    index_col=False
)

## Question 1

> What month/day/year did the attack take place? For example, 09/05/2021

In [2]:
df["DateOnly"] = pd.to_datetime(df["DateTime"]).dt.date
df.groupby("DateOnly").size().sort_values(ascending=False).iloc[:1]


DateOnly
2022-12-24    3540
dtype: int64

Date has to be formatted to match the requested date format

## Question 2

> An attacker got a secret from a file. What was the original file's name?

In [3]:
q2df = df[ (df["DateOnly"] == pd.Timestamp("2022-12-24")) & (df["Category"].str.contains("Add-Content -Path"))]
q2df[ ["DateTime", "Category"] ]

,DateTime,Category
2592,24.12.2022 12:05:07,Creating Scriptblock text (1 of 1):\n$foo | Add-Content -Path 'Recipe'\n\nScriptBlock ID: 00daf2d0-fd6e-4475-8f90-b99224cbe240\nPath:
2638,24.12.2022 12:04:44,Creating Scriptblock text (1 of 1):\n$foo | Add-Content -Path 'Recipe.txt'\n\nScriptBlock ID: c54e091b-147d-4b65-a84d-87ec7453fbbb\nPath:
2721,24.12.2022 12:04:18,Creating Scriptblock text (1 of 1):\n$foo | Add-Content -Path 'Recipe.txt'\n\nScriptBlock ID: 49f0e51a-bcd9-4a8f-a124-3512606afde7\nPath:
2804,24.12.2022 12:03:55,Creating Scriptblock text (1 of 1):\n$foo | Add-Content -Path 'Recipe.txt'\n\nScriptBlock ID: 8e5b9d7d-e1ff-40bc-8727-49ae9530af02\nPath:
2862,24.12.2022 12:03:22,Creating Scriptblock text (1 of 1):\n$foo | Add-Content -Path 'recipe_updated.txt'\n\nScriptBlock ID: e6033506-603f-4380-9be0-79880b40e95c\nPath:
2948,24.12.2022 12:02:45,"Creating Scriptblock text (1 of 1):\n$foo = Get-Content .\Recipe| % {$_-replace 'honey','fish oil'} $foo | Add-Content -Path 'recipe_updated.txt'\n\nScriptBlock ID: 6ea311a5-b9d2-40a6-a794-60e3f9539005\nPath:"
3009,24.12.2022 12:01:20,"Creating Scriptblock text (1 of 1):\n$foo = Get-Content .\Recipe| % {$_ -replace 'honey', 'fish oil'} $foo | Add-Content -Path 'recipe_updated.txt'\n\n\nScriptBlock ID: 1189c065-869d-4386-8386-456c7c89130f\nPath:"


Answer is **Recipe.txt**

## Question 3

> The contents of the previous file were retrieved, changed, and stored to a variable by the attacker. This was done multiple times. Submit the last full Powershell line that performed on these actions

In [4]:
q3df = df[ (df["DateOnly"] == pd.Timestamp("2022-12-24")) & (df["Category"].str.contains("\$foo\s=\sGet-Content", regex=True))].sort_values(by="DateTime", ascending=False).iloc[:1]
q3df["Category"]

2689    Creating Scriptblock text (1 of 1):\n$foo = Get-Content .\Recipe| % {$_ -replace 'honey', 'fish oil'}\n\nScriptBlock ID: 11c6e378-2879-460e-bcd9-06cdfa28c70a\nPath: 
Name: Category, dtype: object

## Question 4

> After storing the altered file contents into the variable, the attacker used the variable 
to run a separate command that wrote the modified data to a file. This was done multiple 
times. Submit the last full PowerShell line that performed only this action.

In [5]:
q4df = df[ (df["DateOnly"] == pd.Timestamp("2022-12-24")) & (df["Category"].str.contains("\$foo \|", regex=True))].sort_values(by="DateTime", ascending=False).iloc[:1]
re.search(r"\n(.+)\n\n", q4df["Category"].iloc[0]).group(1)

"$foo | Add-Content -Path 'Recipe'"

## Question 5

The attacker ran the previous command against a file multiple times. What is the name of this file?

In [6]:
q5df = df[ (df["DateOnly"] == pd.Timestamp("2022-12-24")) & (df["Category"].str.contains("foo", regex=True))].sort_values(by="DateTime", ascending=False)
q5df[ ["DateTime", "Category"] ]

,DateTime,Category
2592,24.12.2022 12:05:07,Creating Scriptblock text (1 of 1):\n$foo | Add-Content -Path 'Recipe'\n\nScriptBlock ID: 00daf2d0-fd6e-4475-8f90-b99224cbe240\nPath:
2638,24.12.2022 12:04:44,Creating Scriptblock text (1 of 1):\n$foo | Add-Content -Path 'Recipe.txt'\n\nScriptBlock ID: c54e091b-147d-4b65-a84d-87ec7453fbbb\nPath:
2689,24.12.2022 12:04:37,"Creating Scriptblock text (1 of 1):\n$foo = Get-Content .\Recipe| % {$_ -replace 'honey', 'fish oil'}\n\nScriptBlock ID: 11c6e378-2879-460e-bcd9-06cdfa28c70a\nPath:"
2721,24.12.2022 12:04:18,Creating Scriptblock text (1 of 1):\n$foo | Add-Content -Path 'Recipe.txt'\n\nScriptBlock ID: 49f0e51a-bcd9-4a8f-a124-3512606afde7\nPath:
2772,24.12.2022 12:04:10,"Creating Scriptblock text (1 of 1):\n$foo = Get-Content .\Recipe| % {$_-replace 'honey','fish oil'}\n\nScriptBlock ID: e7266037-4244-4d41-96cc-9d2488d84bde\nPath:"
2804,24.12.2022 12:03:55,Creating Scriptblock text (1 of 1):\n$foo | Add-Content -Path 'Recipe.txt'\n\nScriptBlock ID: 8e5b9d7d-e1ff-40bc-8727-49ae9530af02\nPath:
2862,24.12.2022 12:03:22,Creating Scriptblock text (1 of 1):\n$foo | Add-Content -Path 'recipe_updated.txt'\n\nScriptBlock ID: e6033506-603f-4380-9be0-79880b40e95c\nPath:
2913,24.12.2022 12:03:10,"Creating Scriptblock text (1 of 1):\n$foo = Get-Content .\Recipe| % {$_-replace 'honey','fish oil'}\n\nScriptBlock ID: b039d71f-52cc-41b7-acf6-a8ca43ad5035\nPath:"
2948,24.12.2022 12:02:45,"Creating Scriptblock text (1 of 1):\n$foo = Get-Content .\Recipe| % {$_-replace 'honey','fish oil'} $foo | Add-Content -Path 'recipe_updated.txt'\n\nScriptBlock ID: 6ea311a5-b9d2-40a6-a794-60e3f9539005\nPath:"
3009,24.12.2022 12:01:20,"Creating Scriptblock text (1 of 1):\n$foo = Get-Content .\Recipe| % {$_ -replace 'honey', 'fish oil'} $foo | Add-Content -Path 'recipe_updated.txt'\n\n\nScriptBlock ID: 1189c065-869d-4386-8386-456c7c89130f\nPath:"


The answer is **Recipe.txt**

## Question 6

> Where any files deleted? (Yes/No)

Answer is **Yes**

# Question 7

> Was the original file (from question 2) deleted? (Yes/No)

The answer is **Yes**

## Question 8

> What is the Event ID of the log that shows the actual command line used to delete the file? 

In [7]:
q8df = df[ (df["DateOnly"] == pd.Timestamp("2022-12-24")) & (df["Category"].str.contains("Recipe.txt"))].groupby("EventID").size().sort_values(ascending=True)
q8df


EventID
4104    4
4103    7
dtype: int64

The answer is **4104**

## Question 9

> Is the secret ingredient compromised? (Yes/No)

The answer is **Yes**

## Question 10

> What is the secret ingredient?

In [8]:
q10dfa = df[ (df["DateOnly"] == pd.Timestamp("2022-12-24")) & (df["Category"].str.contains("\$foo\s=\sGet-Content", regex=True))].sort_values(by="DateTime", ascending=False).iloc[:1]
q10dfa[["Category"]]

,Category
2689,"Creating Scriptblock text (1 of 1):\n$foo = Get-Content .\Recipe| % {$_ -replace 'honey', 'fish oil'}\n\nScriptBlock ID: 11c6e378-2879-460e-bcd9-06cdfa28c70a\nPath:"


From this query, from question 3, we saw that the word "honey" was replaced with "fish oil". 

In [9]:
q10dfb = df[ (df["DateOnly"] == pd.Timestamp("2022-12-24")) & (df["Category"].str.contains("honey|Honey", regex=True))].sort_values(by="DateTime", ascending=False).iloc[:1]
q10dfb

Level             DateTime                        Origin  EventID  \
2542  Informasjon  24.12.2022 12:05:23  Microsoft-Windows-PowerShell     4103   

                    Task  \
2542  Executing Pipeline   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

By simply querying for the word "honey" we found an interesting sentence: **1/2 tsp honey (secret ingredient)**